# Step 1. Open the data file and read the general information

## Project description
For a mobile carrier, Megaline, we want to develop a model that would analyze subscribers' behavior and recommend one of Megaline's newer plans: Smart or Ultra. We need to develop a model that will pick the right plan. We will try to develop a model with the highest possible accuracy. In this project, the threshold for accuracy is 0.75. We will check the accuracy using the test dataset.

## Import

In [116]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

## Load data

In [117]:
try:
    df = pd.read_csv('users_behavior.csv')
except:
    df = pd.read_csv('datasets/users_behavior.csv')

## Step 2. Check the data

- сalls — number of calls,
- minutes — total call duration in minutes,
- messages — number of text messages,
- mb_used — Internet traffic used in MB,
- is_ultra — plan for the current month (Ultra - 1, Smart - 0).

In [118]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


No missing values

In [119]:
df.sample(5)

,calls,minutes,messages,mb_used,is_ultra
1399,50.0,359.73,102.0,3664.60,1
2454,30.0,225.11,30.0,12280.81,0
2806,107.0,733.45,98.0,10756.78,1
2553,66.0,479.26,20.0,11500.33,0
2209,15.0,84.66,13.0,4353.73,1


In [120]:
df['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

In [121]:
df['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

More then double amount of Smart then Ultra plan. We should remember to see that the split for validation keeps the same proportion and not worse. We will create a function to calculate this proportion:

In [122]:
def proportion_of_ultra(df):
    value_count = df['is_ultra'].value_counts()
    proportion = df['is_ultra'].value_counts()[1]/(
        df['is_ultra'].value_counts()[1]+df['is_ultra'].value_counts()[0])
    return round(proportion, 2)

# Split the source data into a training set, a validation set, and a test set.

In [123]:
df_train, df_valid_and_test = train_test_split(df, test_size=0.4, random_state=12345)
df_valid, df_test = train_test_split(df_valid_and_test, test_size=0.5, random_state=12345)

See if the proportion of ultra plan stay the same after split

In [124]:
print('df_train proportion:', proportion_of_ultra(df_train))
print('df_valid proportion:', proportion_of_ultra(df_valid))
print('df_test proportion:', proportion_of_ultra(df_test))

df_train proportion: 0.31
df_valid proportion: 0.29
df_test proportion: 0.32


yes

# Investigate the quality of different models by changing hyperparameters. Briefly describe the findings of the study.

Because our task is to recommend between plans then any wrong recommendation will be considered as error. Whether we decide to recommend for Ultra when the target for this observation was Smart or the opposite. Therefore we will evaluate the model with accuracy metric.

This is a classification task so we will check which learning algorithm for classification yields the best accuracy. The models we will check are: Decision tree, Random forest and Logistic regression

create features and target for the models check

In [125]:
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

## Decision tree

loop tree depth to optimize the model depth with best accuracy

In [126]:
for depth in range(1,6):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) # create a model with the given depth
    model.fit(features_train, target_train) # train the model
    predictions_valid = model.predict(features_valid) # get the model's predictions
    result = accuracy_score(target_valid, predictions_valid) # calculate the accuracy
    print("max_depth =", depth, ": ", end='')
    print(result)

max_depth = 1 : 0.7542768273716952
max_depth = 2 : 0.7822706065318819
max_depth = 3 : 0.7853810264385692
max_depth = 4 : 0.7791601866251944
max_depth = 5 : 0.7791601866251944


Depth 3 give the highest accuracy - 0.785

## Random forest

loop tree number of trees to optimize the model with best accuracy

In [127]:
best_score = 0
best_est = 0
for est in range(1, 10): # choose hyperparameter range
    model = RandomForestClassifier(random_state=12345, n_estimators=est) # set number of trees
    model.fit(features_train, target_train) # train model on training set
    score = model.score(features_valid, target_valid) # calculate accuracy score on validation set
    if score > best_score:
        best_score = score# save best accuracy score on validation set
        best_est = est# save number of estimators corresponding to best accuracy score

print("Accuracy of the best model on the validation set (n_estimators = {}): {}".format(best_est, best_score))

Accuracy of the best model on the validation set (n_estimators = 8): 0.7822706065318819


## Logistic regression

In [128]:
model = LogisticRegression(random_state=12345, solver='lbfgs')
model.fit(features_train, target_train) 
score = model.score(features_valid, target_valid) 
print('The accuracy of the validation set is:', score)

The accuracy of the validation set is: 0.7107309486780715


## Model study conclusion

The highest accuracy we got is 0.785 and the model that allowed it is decision tree with depth of 3. Other models with different hyperparameters couldn't compete that. Therefore we select this model with and will check the quality of this model with the test set

# Check the quality of the model using the test set

In [130]:
model = DecisionTreeClassifier(random_state=12345, max_depth=3)
model.fit(features_train, target_train) # fit with train data
predictions_test = model.predict(features_test) # predict with test data
result = accuracy_score(target_test, predictions_test)
print('The accuracy of the test set is:', result)

The accuracy of the test set is: 0.7791601866251944


The accuracy is a bit lower but very close to what achieved in the validation of the model. We can say that the model is not overfitted and also we can't say it's underfitted because it's still a high accuracy